# Creating pathfinder subsample

June 20, 2022  
Gully & Ryan H.

The goal of this notebook is to make the pathfinder sample.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightkurve as lk
from tqdm import tqdm


sns.set_context('notebook', font_scale=1.5)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
names = ['EPIC','Campaign','Teff','log g','Prot','ΔProt','hpeak','Rvar','Kp','MG']

In [ ]:
df = pd.read_csv('../../data/Reinhold_Hekker2020/table2.dat', 
                 delim_whitespace=True, names=names, na_values='---')

Looks good!  We see the same trend we had in our proposal figure 2.

## Select a subsample of sources

First search for some high amplitude variable stars

In [ ]:
criterion1 = (df.Prot > 1) & (df.Prot < 10)
criterion2 = (df.Rvar > 0.5) & (df.Rvar < 20)
criterion3 = (df.Teff > 4000) & (df.Teff < 4500)
criteria = criterion1 & criterion2 & criterion3

In [ ]:
criteria.sum()

In [ ]:
#plt.plot(df.Prot, df.Rvar, '.', alpha=0.02);
plt.plot(df.Prot[criterion3], df.Rvar[criterion3], '.', alpha=0.1);
plt.plot(df.Prot[criteria], df.Rvar[criteria], '.', alpha=0.5);
#plt.ylim(3e2, 2e5)
plt.xlim(1e0, 1e2)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('$P_{\mathrm{rot}}$')
plt.ylabel('$\propto$ Amplitude (%)')
plt.title('Reinhold & Hekker 2020 Table 2')

In [ ]:
df[criteria].head()

In [ ]:
df_subset=df[criteria].reset_index(drop=True)

In [ ]:
df_subset

## Make a subsub sample

### Prepopulate our columns

In [ ]:
df_subset['N_EVEREST'] = np.NaN
df_subset['N_TESS_SPOC'] = np.NaN
df_subset['Period_TESS'] = 0
df_subset['Amplitude_TESS'] = 0
df_subset['Period_K2'] = 0
df_subset['Amplitude_K2'] = 0
df_subset['Sector'] = np.NaN

In [ ]:
df_tiny = df_subset.head(15)
df_tiny

In [ ]:
df_tiny.iloc[0].to_frame()

### Predownload so that it runs faster later

Let's find one of the sources that *also* has TESS data available

Delete the cell below if you want to run on the entire subset of 400+ sources...

In [ ]:
df_subset = df_tiny

In [ ]:
n_sources = len(df_subset)
n_sources

We want to have at least 1 EVEREST lightcurve and 1 SPOC lightcurve for all sources.

In [ ]:
%%capture
for i in tqdm(range(n_sources)):
    # find the name of the star
    name = 'EPIC ' + df_subset.iloc[i].EPIC.astype(int).astype(str)
    
    # find the TESS lightcurve for the star
    sr = lk.search_lightcurve(name, mission="TESS")
    
    # find the number of TESS missions for the star
    df_subset.loc[i, 'N_TESS_SPOC'] = len(sr)
    
    # check if there is TESS data available
    if len(sr) > 0:
        # download the data for the lightcurve
        lc = sr[0].download()
    
        # remove NaNs and normalize the data
        lc = lc.remove_nans().normalize()
    
        # find the amplitude percentage
        vector = lc.flux.value
        lo, hi = np.percentile(vector, (5, 95))
        peak_to_valley = hi-lo
    
        # add the amplitude to the data table
        df_subset.loc[i, 'Amplitude_TESS'] = peak_to_valley
        
        # change the lightcurve into a periodogram and find its period
        period = float(lc.to_periodogram().period_at_max_power.to_value())

        # add the period to the data table
        df_subset.loc[i, 'Period_TESS'] = period
        
#----------------------------------------------------------------------------
# Now for the Kepler data

        
    # find the Kepler lightcurve for the star
    sr = lk.search_lightcurve(name, mission="K2", author='EVEREST')
    
    # find the number of Kepler missions for the star
    df_subset.loc[i, 'N_EVEREST'] = len(sr)
    
    # check if there is Kepler data available
    if len(sr) > 0:
        # download the data for the lightcurve
        lc = sr[0].download()
    
        # remove NaNs and normalize the data
        lc = lc.remove_nans().normalize()

        # find the amplitude percentage
        vector = lc.flux.value
        lo, hi = np.percentile(vector, (5, 95))
        peak_to_valley = hi-lo

        # add the data to the table
        df_subset.loc[i, 'Amplitude_K2'] = peak_to_valley

        # change the lightcurve into a periodogram and find its period
        period = float(lc.to_periodogram().period_at_max_power.to_value())

        # add the period to the data table
        df_subset.loc[i, 'Period_K2'] = period
    
    # Todo 
    ## Populate the Amplitde and Period columns
    # df_subset.loc[i,'Amplitude'] = ...
    # df_subset.loc[i,'Period'] = ...

In [ ]:
df_subset

In [ ]:
df_subset.to_csv('pathfinder_sample.csv', index=False)

## Spot check one source...

In [ ]:
name = 'EPIC 202059229'

# find the TESS lightcurve for the star
sr = lk.search_lightcurve(name, author="EVEREST")

# check if there is TESS data available
if len(sr) > 0:
    # download the data for the lightcurve
    lc = sr[0].download()

    # remove NaNs and normalize the data
    lc = lc.remove_nans().normalize()

    # find the amplitude percentage
    vector = lc.flux.value
    lo, hi = np.percentile(vector, (5, 95))
    peak_to_valley = hi-lo

    # add the data to the table
    df_subset.loc[i, 'Amplitude_TESS'] = float(peak_to_valley)
    
    period = float(lc.to_periodogram().period_at_max_power.to_value())
    

In [ ]:
period

In [ ]:
lc.to_periodogram().plot(view='period', scale='log')

### Example manipulations ...

In [ ]:
sr.table.to_pandas()

In [ ]:
mask = (sr.table['author'] == 'EVEREST').data
mask.sum()

In [ ]:
sr = lk.search_lightcurve("EPIC 202059229", mission='TESS')
sr
lc = sr[0].download()

In [ ]:
lc = lc.remove_nans().normalize()

In [ ]:
vector = lc.flux.value
vector

In [ ]:
lo, hi = np.percentile(vector, (5, 95))

In [ ]:
lo, hi

In [ ]:
ax = lc.normalize().plot()
ax.axhline(hi)
ax.axhline(lo)

In [ ]:
peak_to_valley = hi-lo
peak_to_valley

In [ ]:
type(sr)

In [ ]:
df_sr = sr.table.to_pandas()

In [ ]:
df_subset

In [ ]:
len(sr)

In [ ]:
sr = lk.search_lightcurve("EPIC 211071889", author="EVEREST", mission="K2")
sr

In [ ]:
lc_K2 = sr.download()

In [ ]:
sr = lk.search_lightcurve("EPIC 211071889", author="SPOC", mission="TESS")
sr[0]

In [ ]:
lc_TESS = sr[0].download()

In [ ]:
scalar = np.percentile(lc_K2.flux, 98)
lc_K2 = lc_K2/scalar
ax = lc_K2.plot()
ax.axhline(1.0, linestyle='dashed')
ax.axhline(0.93, linestyle='dotted', color='#d35400', label='7 % flux loss')
ax.set_title('K2 data')
ax.set_ylim(0.8, 1.1)
ax.legend()

In [ ]:
lc_TESS = lc_TESS.remove_nans().bin(binsize=5)
scalar = np.nanpercentile(lc_TESS.flux, 98)
lc_TESS = lc_TESS/scalar

In [ ]:
ax = lc_TESS.plot()
ax.axhline(1.0, linestyle='dashed')
ax.axhline(0.93, linestyle='dotted', color='#d35400', label='7 % flux loss')
ax.axhline(0.955, linestyle='solid', color='#2ecc71', label='4.5 % flux loss')
ax.set_title('TESS data')
ax.set_ylim(0.8, 1.1)
ax.legend(fontsize=12)

In [ ]:
assert len(sr) == 1

In [ ]:
lc=sr.download()

In [ ]:
lc = lc.remove_outliers(sigma=4,sigma_upper=3).normalize()

In [ ]:
pg = lc.to_periodogram(nterms=5)

In [ ]:
ax = pg.plot(view='period', scale='log')
ax.axvline(pg.period_at_max_power.value, linestyle='dotted', label=f'{pg.period_at_max_power:0.5f}')
ax.axvline(6.70, linestyle='dashed', label='6.7 d (Reinhold & Hekker 2020)', color = 'red')
ax.legend(fontsize = 12)

In [ ]:
pg.period_at_max_power

In [ ]:
ax = lc.plot()
pg.model(lc.time).plot(ax=ax)
pg.model(lc.time, frequency=pg.frequency_at_max_power/2).plot(ax=ax)